In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install implicit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 61.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import scipy.sparse as sp
from scipy.sparse import csr_matrix

import implicit

In [ ]:
PATH = '/content/drive/MyDrive/Study/MIPT_magistery/semester2/hackathon'

In [ ]:
df = pd.read_parquet(f'{PATH}/data/train_mfti.parquet')

# Предобработка данных

In [ ]:
# куки в первоначальном виде оставим в колонке cookie_id_initial
# колонку cookie_id подготовим для дольнейшего обучения
df.rename(columns = {'cookie_id':'cookie_id_initial'}, inplace = True )
df['cookie_id'] = df['cookie_id_initial']

In [ ]:
# создадим датафрейм в котором посчитаем количество разных cookie_id для одного и тгго же user_id
df2 = df.groupby('user_id').agg({'cookie_id' :'nunique'})
df2.rename(columns = {'cookie_id':'cookie_count'}, inplace = True )

In [ ]:
# видим, что у нас есть 7522 пользователя которые, по всей видимости, заходят с разных устройств
df2[df2.cookie_count>1]

,cookie_count
user_id,
0011a0b25c7b46fdac25131344f9ec75,2
00128b91d8c04cd39719fcc0dbc432b0,2
0019cc2d997d43efbb2edd4551609e49,2
002807eab30748bd9f9b0fc6736b4218,2
003ca2d411994efebcb808c39b412a95,2
...,...
ffde525800e74db3aec78505ae90dd49,2
ffe1f9ea30a84cad8375a03054768ae2,2
ffedab7762c64401a0f058c2ed1fe0d4,2


In [ ]:
# Присоединим эту таблицу к основной
# Заполним пропуски единицами, так как для этих cookie_id нет информации по user_id	
df = pd.merge(df, df2, on='user_id', how='left')
df['cookie_count'] = df['cookie_count'].fillna(1)
df['cookie_count'] = df['cookie_count'].astype(int)

In [ ]:
del df2

In [ ]:
# Заменим id в колонке cookie_id на одинаковые для пользователей с одинаковым user_id и cookie_count>1
# В качестве значения для замены возьмем user_id с припиской _changed
df.loc[(df['cookie_count']>1), 'cookie_id'] = df['user_id'] + "_changed"


In [ ]:
df = df.drop_duplicates()

In [ ]:
df[df.cookie_count>1]

,event_date,event_timestamp,vacancy_id_,cookie_id_initial,user_id,event_type,cookie_id,cookie_count
78,2022-08-01,1659360049,255199,ae0baf127c6d472fa228cba9fb662c2e,b01e6b595dc1411095b4698414d2ea32,show_vacancy,b01e6b595dc1411095b4698414d2ea32_changed,3
79,2022-08-01,1659360024,102608,ae0baf127c6d472fa228cba9fb662c2e,b01e6b595dc1411095b4698414d2ea32,show_vacancy,b01e6b595dc1411095b4698414d2ea32_changed,3
80,2022-08-01,1659360012,102608,ae0baf127c6d472fa228cba9fb662c2e,b01e6b595dc1411095b4698414d2ea32,show_vacancy,b01e6b595dc1411095b4698414d2ea32_changed,3
81,2022-08-01,1659359993,102608,ae0baf127c6d472fa228cba9fb662c2e,b01e6b595dc1411095b4698414d2ea32,show_vacancy,b01e6b595dc1411095b4698414d2ea32_changed,3
82,2022-08-01,1659359964,128055,ae0baf127c6d472fa228cba9fb662c2e,b01e6b595dc1411095b4698414d2ea32,show_vacancy,b01e6b595dc1411095b4698414d2ea32_changed,3
...,...,...,...,...,...,...,...,...
12292154,2022-08-11,1660205708,197331,4049280d091b4428969dae790a5ec10d,1140b1457b8f4b379b6e59211306d70e,preview_click_response,1140b1457b8f4b379b6e59211306d70e_changed,2
12292232,2022-08-11,1660189405,164559,bd975029968c467ea1514808fba6e18c,89eae7724a6943eb806a4c215e48ab92,preview_click_response,89eae7724a6943eb806a4c215e48ab92_changed,2
12292233,2022-08-11,1660231897,154469,3b98487d9346482a86450c60afa3f1e5,62b6b8926dea45d39c820346e1070ca1,preview_click_response,62b6b8926dea45d39c820346e1070ca1_changed,3
12292269,2022-08-11,1660195723,243355,4cfcfa71e8224b9b9b8bd9cbeab38dce,0646362876664676b2d435b24f23c5b7,preview_click_response,0646362876664676b2d435b24f23c5b7_changed,2


In [ ]:
dict_to_final_mapping_cookie_id = dict(df[['cookie_id_initial', 'cookie_id']].values)

Попробуем в финальных рекомендациях отбросить неактивные какое-то время вакансии (может их непопулярность - неспроста?)

In [ ]:
vacancies_with_last_interact = df.groupby('vacancy_id_')['event_date'].max().to_dict()

In [ ]:
min(vacancies_with_last_interact.items()), max(vacancies_with_last_interact.items())

((100001, '2022-09-29'), (260167, '2022-10-01'))

In [ ]:
not_active_more_than_one_day = []
not_active_more_than_two_days = []

for vacancy, last_act_date in vacancies_with_last_interact.items():
    if last_act_date == '2022-09-29':
        not_active_more_than_two_days.append(vacancy)
    if last_act_date <= '2022-09-30':
        not_active_more_than_one_day.append(vacancy)

len(not_active_more_than_one_day), len(not_active_more_than_two_days)

(140242, 9461)

# Part 1

Превратим исходные данные в таблицу различных паттернов действий пользователей

In [ ]:
df_coo = df.pivot_table(values='event_timestamp', index=['cookie_id', 'vacancy_id_'], columns='event_type', aggfunc='count')
df_coo = df_coo.fillna(0)
df_coo[df_coo > 1] = 1
df_coo

event_type                                    click_contacts  click_favorite  \
cookie_id                        vacancy_id_                                   
0000c4548c3944c08972bbdc1fa4eb85 137659                  0.0             0.0   
                                 153975                  0.0             0.0   
                                 174953                  0.0             1.0   
                                 176171                  0.0             0.0   
                                 182445                  0.0             0.0   
...                                                      ...             ...   
ffffdb17f45b4032b386d691d52e6c00 107125                  1.0             0.0   
                                 107893                  0.0             0.0   
                                 150843                  0.0             0.0   
                                 160164                  0.0             0.0   
                                 185412                  1.0             0.0   

event_type                                    click_phone  click_response  \
cookie_id                        vacancy_id_                                
0000c4548c3944c08972bbdc1fa4eb85 137659               0.0             0.0   
                                 153975               0.0             0.0   
                                 174953               0.0             0.0   
                                 176171               0.0             0.0   
                                 182445               0.0             0.0   
...                                                   ...             ...   
ffffdb17f45b4032b386d691d52e6c00 107125               0.0             0.0   
                                 107893               0.0             0.0   
                                 150843               0.0             0.0   
                                 160164               0.0             0.0   
                                 185412               0.0             0.0   

event_type                                    preview_click_contacts  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                          0.0   
                                 153975                          0.0   
                                 174953                          0.0   
                                 176171                          0.0   
                                 182445                          0.0   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                          0.0   
                                 107893                          1.0   
                                 150843                          1.0   
                                 160164                          1.0   
                                 185412                          0.0   

event_type                                    preview_click_favorite  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                          0.0   
                                 153975                          0.0   
                                 174953                          0.0   
                                 176171                          0.0   
                                 182445                          0.0   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                          0.0   
                                 107893                          0.0   
                                 150843                          0.0   
                                 160164                          0.0   
                                 185412                          0.0   

event_type                                    preview_click_phone  \
cookie_id     

Проанализировав каждый паттерн дадим оценку возможного следующего действия пользователя в каждом отдельном взаимодействии

In [ ]:
df_coo = df_coo.reset_index()
df_index = df_coo[['cookie_id', 'vacancy_id_']]
values_cols = ['click_response', 'preview_click_response', 
                'click_contacts', 'click_favorite',
                'click_phone', 'preview_click_contacts',
                'preview_click_favorite', 'preview_click_phone',
                'preview_click_vacancy', 'show_vacancy']
               
display(df_index.head(3))

event_type,cookie_id,vacancy_id_
0,0000c4548c3944c08972bbdc1fa4eb85,137659
1,0000c4548c3944c08972bbdc1fa4eb85,153975
2,0000c4548c3944c08972bbdc1fa4eb85,174953


In [ ]:
user_item_data = csr_matrix(df_coo[values_cols].values)

Bayesian Personalized Ranking (BPR) - это модель рекомендаций, которая используется для ранжирования элементов в персонализированном списке рекомендаций. BPR использует методы байесовского вывода для обучения модели, которая может ранжировать элементы в списке на основе предпочтений пользователя. Он был представлен в 2009 году статьей "BPR: Bayesian Personalized Ranking from Implicit Feedback".

Для обучения модели BPR использует неявную обратную связь, чтобы предсказать, какой элемент пользователь предпочтет. Обратная связь считается неявной, потому что она не содержит явной оценки для элемента. 

BPR использует матричное разложение для извлечения скрытых признаков элементов и пользователей. Он моделирует относительные ранги элементов в списке рекомендаций, а не абсолютные оценки.

Преимущества использования BPR включают:

- BPR может работать с неявной обратной связью, что делает его более универсальным и полезным для больших наборов данных.
- BPR может работать с большим количеством элементов в списке рекомендаций, что делает его особенно полезным для списков рекомендаций, которые могут иметь тысячи или миллионы элементов.

Однако, BPR также имеет свои недостатки:

- BPR не использует явных оценок элементов, что может привести к менее точным рекомендациям, особенно для новых пользователей, у которых еще нет истории покупок.
- BPR может быть более сложным для понимания и реализации, чем другие модели рекомендаций.

In [ ]:
model = implicit.bpr.BayesianPersonalizedRanking(150)
model.fit(user_item_data)

  0%|          | 0/100 [00:00<?, ?it/s]

После обучения модели собираем обратно матрицу действий пользователя, добавив в нее потенциальные действия, полученные на прошлом шаге (гиперпараметр N=4 указывает на то, что нас интересуют только топ 4 предполагаемых интересов пользователя, при этом мы устанавливаем флаг filter_already_liked_items=False, чтобы не потерять информацию об уже совершенных действиях)

Образец получаемой рекомендации
image.png

In [ ]:
data = []
for i in tqdm(df_coo[values_cols].index):
    userid = i
    recommendations = model.recommend(userid,
                                      user_item_data[userid],
                                      N=6,
                                      filter_already_liked_items=False)
    pred_values = [0] * 10
    for i in recommendations[0]:
        pred_values[i] = 1
    data.append(pred_values)

  0%|          | 0/4667335 [00:00<?, ?it/s]

In [ ]:
df_after_BPR = pd.DataFrame(data=data, columns=values_cols)
df_after_BPR[['cookie_id', 'vacancy_id_']] = df_index[['cookie_id', 'vacancy_id_']]
new_order = [ 'cookie_id', 'vacancy_id_', 
             'click_response', 'preview_click_response', 
             'click_contacts', 'preview_click_contacts',
             'click_phone', 'preview_click_phone', 
             'click_favorite', 'preview_click_favorite', 
             'show_vacancy', 'preview_click_vacancy',
        ]
df_after_BPR = df_after_BPR[new_order]
df_after_BPR = df_after_BPR.set_index(['cookie_id', 'vacancy_id_'])
df_after_BPR

click_response  \
cookie_id                        vacancy_id_                   
0000c4548c3944c08972bbdc1fa4eb85 137659                    1   
                                 153975                    1   
                                 174953                    1   
                                 176171                    1   
                                 182445                    1   
...                                                      ...   
ffffdb17f45b4032b386d691d52e6c00 107125                    1   
                                 107893                    0   
                                 150843                    0   
                                 160164                    0   
                                 185412                    1   

                                              preview_click_response  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                            1   
                                 153975                            1   
                                 174953                            1   
                                 176171                            1   
                                 182445                            1   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                            1   
                                 107893                            1   
                                 150843                            1   
                                 160164                            1   
                                 185412                            1   

                                              click_contacts  \
cookie_id                        vacancy_id_                   
0000c4548c3944c08972bbdc1fa4eb85 137659                    0   
                                 153975                    1   
                                 174953                    0   
                                 176171                    1   
                                 182445                    1   
...                                                      ...   
ffffdb17f45b4032b386d691d52e6c00 107125                    1   
                                 107893                    1   
                                 150843                    1   
                                 160164                    1   
                                 185412                    0   

                                              preview_click_contacts  \
cookie_id                        vacancy_id_                           
0000c4548c3944c08972bbdc1fa4eb85 137659                            0   
                                 153975                            0   
                                 174953                            0   
                                 176171                            0   
                                 182445                            0   
...                                                              ...   
ffffdb17f45b4032b386d691d52e6c00 107125                            0   
                                 107893                            1   
                                 150843                            1   
                                 160164                            1   
                                 185412                            0   

                                              click_phone  \
cookie_id                        vacancy_id_                
0000c4548c3944c08972bbdc1fa4eb85 137659                 0   
                                 153975                 0   
                                 174953                 0   
                                 176171                 0   
                                 182445                 0   
...                                                   ...   
ffffdb17f45b4032b3

# Part 2

Так как по значимости все действия равноценны, то присвоим каждому оценку 1.
Для определения отношения пользователя к конкретной вакансии вцелом, мы просто сложим оценки всех значимых для нас действий

В итоге у нас получится матрица отношения пользователя к каждой вакансии, с которой ему пришлось взаимодействовать. При чем с учётом предположения о возможном будущем взаимодействии с этой вакансией

In [ ]:
# marking

df_train = df_after_BPR.copy()

most_important_cols = [
                    'click_response', 'preview_click_response',
                    'click_contacts', 'preview_click_contacts',
                    'click_phone', 'preview_click_phone',
                    # 'click_favorite', 'preview_click_favorite', 
                    # 'show_vacancy', 'preview_click_vacancy',
                    ]

least_important_cols = [
                    'click_favorite', 'preview_click_favorite', 
                    'show_vacancy', 'preview_click_vacancy',
                    ]


df_train['mark'] = df_train[most_important_cols].sum(axis=1)
df_train = df_train['mark'].reset_index()

dict_cookie_id = {}
i = 0
for cookie_id in df_train['cookie_id'].unique():
    dict_cookie_id[cookie_id] = i
    i += 1

df_train['cookie_id'] = df_train['cookie_id'].apply(lambda x: dict_cookie_id[x])
df_train

,cookie_id,vacancy_id_,mark
0,0,137659,2
1,0,153975,3
2,0,174953,2
3,0,176171,3
4,0,182445,3
...,...,...,...
4667330,325737,107125,3
4667331,325737,107893,4
4667332,325737,150843,4
4667333,325737,160164,4


In [ ]:
df_train['mark'].min(), df_train['mark'].max(), df_train['mark'].unique()

(2, 5, array([2, 3, 5, 4]))

Далее мы создаём вновь спарс матрицу и обучаем ее на алгоритме ALS, после чего проверяем на точность предсказания топ-5 рекомендованных вакансий (используемая метрика precision@5)

In [ ]:
data = df_train['mark']
row = df_train['cookie_id']
col = df_train['vacancy_id_']
# convert DataFrame to COO sparse matrix
user_item_data = sp.coo_matrix((data, (row, col))).tocsr()

In [ ]:
df_test = pd.read_parquet(f'{PATH}/data/test_public_mfti.parquet')
df_test.head()

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"


In [ ]:
def precision_at_5(model, df_test, activity_filter_out):
    precision = []
    for idx, row in df_test.iterrows():
        cookie_id = dict_to_final_mapping_cookie_id[row['cookie_id']]
        userid = dict_cookie_id[cookie_id]
        recommendations = model.recommend(userid, 
                                          user_item_data[userid], 
                                          filter_items=activity_filter_out,
                                          N=5)[0]
        TP = len(set(recommendations).intersection(set(row['vacancy_id_'][:])))
        precision.append(TP / 5)

    return np.mean(precision)

result_grid = {
    'factors': [],
    'regularization': [],
    'alpha': [],
    'precision@5': [],
}

def train(factors, regularization, alpha, activity_filter_out=not_active_more_than_one_day):
    model = implicit.als.AlternatingLeastSquares(
                                        factors=factors,
                                        regularization=regularization,
                                        alpha=alpha,
                                        calculate_training_loss=True, 
                                        iterations=25,
                                        random_state=42)
    model.fit(user_item_data)
    precision = precision_at_5(model, df_test, activity_filter_out)
    
    result_grid['factors'].append(factors)
    result_grid['regularization'].append(regularization)
    result_grid['alpha'].append(alpha)
    result_grid['precision@5'].append(precision)
    print(f'precision@5: {precision}')

In [ ]:
i = 1
for factors in [400, 500, 600]:
    for regularization in [0.01, 0.1]:
        for alpha in [1.0, 5.0]:
            print(i, 'factors', factors,
                  'regularization', regularization,
                  'alpha', alpha)
            train(factors, regularization, alpha)
            print('-'*10)
            i += 1

1 factors 400 regularization 0.01 alpha 1.0


  0%|          | 0/25 [00:00<?, ?it/s]

precision@5: 0.0572538860103627
----------
2 factors 400 regularization 0.01 alpha 5.0


  0%|          | 0/25 [00:00<?, ?it/s]

precision@5: 0.0572538860103627
----------
3 factors 400 regularization 0.1 alpha 1.0


...

precision@5: 0.05310880829015544
----------
11 factors 600 regularization 0.1 alpha 1.0


  0%|          | 0/25 [00:00<?, ?it/s]

precision@5: 0.0533678756476684
----------
12 factors 600 regularization 0.1 alpha 5.0


  0%|          | 0/25 [00:00<?, ?it/s]

precision@5: 0.0533678756476684
----------


In [ ]:
result = pd.DataFrame(result_grid).sort_values(by=['precision@5'], ascending=False)
result

,factors,regularization,alpha,precision@5
0,400,0.01,1.0,0.057254
1,400,0.01,5.0,0.057254
2,400,0.10,1.0,0.056736
3,400,0.10,5.0,0.056736
10,600,0.10,1.0,0.053368
11,600,0.10,5.0,0.053368
4,500,0.01,1.0,0.053109
5,500,0.01,5.0,0.053109
6,500,0.10,1.0,0.053109
7,500,0.10,5.0,0.053109


# Результат
### Образец

In [ ]:
pd.read_parquet(f'{PATH}/data/test_private_sample_submission_mfti.parquet')

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [ ]:
private_test_df = pd.read_parquet(f'{PATH}/data/test_private_users_mfti.parquet')
private_test_df

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a
...,...
3081,ffadd195859444d2ade2479b0611c5c1
3082,ffbc08b528c64f22996873fc63872202
3083,ffdeaf3c34544529880aebf17c103f6c
3084,ffefa79a74804ee69e6c131e0d05b948


In [ ]:
best_features = result.head(1).iloc[0]
print(best_features)

model = implicit.als.AlternatingLeastSquares(
                                factors=best_features['factors'],
                                regularization=best_features['regularization'],
                                alpha=best_features['alpha'],
                                calculate_training_loss=True, 
                                iterations=25,
                                random_state=42)
model.fit(user_item_data)

factors           400.000000
regularization      0.010000
alpha               1.000000
precision@5         0.057254
Name: 0, dtype: float64


  0%|          | 0/25 [00:00<?, ?it/s]

### Рекомендации

In [ ]:
rec_dict = {
    'cookie_id': [],
    'predictions': []    
}

for idx, row in private_test_df.iterrows():
    cookie_id = dict_to_final_mapping_cookie_id[row['cookie_id']]
    userid = dict_cookie_id[cookie_id]
    recommendations = model.recommend(userid, 
                                          user_item_data[userid], 
                                          filter_items=not_active_more_than_one_day,
                                          N=5)[0]
    rec_dict['cookie_id'].append(row['cookie_id'])
    rec_dict['predictions'].append(list(recommendations))

recommendation_df = pd.DataFrame(rec_dict)
recommendation_df

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[182382, 111848, 136514, 138652, 253440]"
1,0035c298d8c64f368ae730a9cca9bb20,"[254514, 102516, 121955, 253549, 214466]"
2,00956458877448ec9fba87fb97443fdf,"[219770, 253678, 246285, 101462, 259050]"
3,0099387c921b41e7bae6c99dd8254b60,"[173015, 135584, 202969, 194369, 197407]"
4,009f65e8ae99413a8da94a491320580a,"[120188, 102693, 109360, 121955, 106881]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[127706, 106943, 130474, 253946, 153970]"
3082,ffbc08b528c64f22996873fc63872202,"[206907, 235614, 214245, 108098, 112819]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[216952, 113294, 213822, 215249, 117605]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[220718, 212953, 237288, 242560, 246815]"


In [ ]:
recommendation_df.to_parquet('submission_mfti_BBB.parquet', index=False)
pd.read_parquet('submission_mfti_BBB.parquet')

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[182382, 111848, 136514, 138652, 253440]"
1,0035c298d8c64f368ae730a9cca9bb20,"[254514, 102516, 121955, 253549, 214466]"
2,00956458877448ec9fba87fb97443fdf,"[219770, 253678, 246285, 101462, 259050]"
3,0099387c921b41e7bae6c99dd8254b60,"[173015, 135584, 202969, 194369, 197407]"
4,009f65e8ae99413a8da94a491320580a,"[120188, 102693, 109360, 121955, 106881]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[127706, 106943, 130474, 253946, 153970]"
3082,ffbc08b528c64f22996873fc63872202,"[206907, 235614, 214245, 108098, 112819]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[216952, 113294, 213822, 215249, 117605]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[220718, 212953, 237288, 242560, 246815]"
